In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

# Naive Approach

In [ ]:
model_path = '/Users/renjie.tan/Workspace/selfie_quality_check/checkpoints/mixed_models/mix_v0.16_mobilenetv2_224.h5'

In [ ]:
keras_model = tf.keras.models.load_model(model_path)
keras_model.summary()

In [ ]:
pb_save_path = '/Users/renjie.tan/Workspace/selfie_quality_check/checkpoints/mixed_models/mix_v016_mobilenetv2_244'
# keras_model.save(pb_save_path)

In [ ]:
pb_model = tf.keras.models.load_model(pb_save_path)

In [ ]:
pb_model.summary()

# Another approach

In [ ]:
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

In [ ]:
full_model = tf.function(lambda x: keras_model(x))
full_model = full_model.get_concrete_function(
    tf.TensorSpec(keras_model.inputs[0].shape, keras_model.inputs[0].dtype))

In [ ]:
# Get frozen graph def
frozen_func = convert_variables_to_constants_v2(full_model)
frozen_func.graph.as_graph_def()

In [ ]:
layers = [op.name for op in frozen_func.graph.get_operations()]
print("-" * 60)
print("Frozen model layers: ")
for layer in layers:
    print(layer)
print("-" * 60)
print("Frozen model inputs: ")
print(frozen_func.inputs)
print("Frozen model outputs: ")
print(frozen_func.outputs)

In [ ]:
frozen_graph_filename = 'mix_v016_mobilenetv2_244'

In [ ]:
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir=pb_save_path,
                  name=f"{frozen_graph_filename}.pb",
                  as_text=False)
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir=pb_save_path,
                  name=f"{frozen_graph_filename}.pbtxt",
                  as_text=True)

# Testing

In [ ]:
test_image_path = '/Users/renjie.tan/Workspace/Silent-Face-Anti-Spoofing/images/sample/image_F1.jpg'

In [ ]:
def read_image(image_path):
    return cv2.resize(cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB), (224,224))

In [ ]:
test_image = read_image(test_image_path)

In [ ]:
plt.imshow(test_image)

In [ ]:
test_batch = np.expand_dims(test_image, axis = 0)

In [ ]:
keras_prediction = keras_model.predict(test_batch)
pb_prediction = pb_model.predict(test_batch)

In [ ]:
keras_prediction

In [ ]:
pb_prediction